
#### 1.1 How many goals were attempted by head
#### 1.2 How many matches had attendees more than 12000
#### 2.1 Shots list where goal was scored outside 16m box sorted by descending order
#### 2.2 Rank teams based on average attendance at home games 

In [1]:
from MySqlStruct import *

@measure_execution_time
def MySqlExecTime():
    return MySql_connect()

print('----Loading MySql----')
result, execution_time = MySqlExecTime()  # Get both the result and execution time

# Unpack the values from the result
cursor,connection,df_shots,df_matches,df_players,df_teams = result

----Loading MySql----
Inserted 245180 rows into shots.
Inserted 14148 rows into matches.
Inserted 20414 rows into all_players.
Inserted 675 rows into teams.

Data inserted successfully for all files!
Execution time: 9.1673 seconds


In [2]:
# print(df_shots.columns)
# print(df_shots.head(5))
# print(df_matches.columns)
# print(df_matches.head(5))
# print(df_players.columns)
# print(df_players.head(5))
# print(df_teams.columns)
# print(df_teams.head(5))


# 1.1 How many goals were attempted by head

In [3]:
# Create index on Outcome and Body Part (do this once)
def create_index_1_1():
    cursor.execute("CREATE INDEX idx_outcome_body_part ON shots (Outcome, `Body Part`);")
    connection.commit()
    print("Index Created: idx_outcome_body_part")

# Function to execute the query with or without index
@measure_execution_time
def MySQLQ1_1(index=False):
    # Create index if needed
    if index:
        create_index_1_1()
        print("Query with index:")
    else:
        print("Query without index:")
    
    # Define the query
    query = "SELECT * FROM shots WHERE Outcome = 'Goal' AND `Body Part` = 'Head';"
    
    # Execute the query
    cursor.execute(query)
    
    # Fetch all the results
    result = cursor.fetchall()
    
    # Print the first result (optional)
    if result:
        print(result[0])
    
    # Print the number of records
    print(f"Goals Scored By Head: {len(result)}")

# Execute the function and measure execution time with and without index
result_no_index, execution_time_no_index = MySQLQ1_1(index=False)
result_with_index, execution_time_with_index = MySQLQ1_1(index=True)

# Compare execution times
print(f'-- Execution is: {execution_time_no_index / execution_time_with_index} times faster with the index --')




Query without index:
(1, '4a1a9578', 'Goal', 8.0, 'Head', 'de515487')
Goals Scored By Head: 3963
Execution time: 0.1380 seconds
Index Created: idx_outcome_body_part
Query with index:
(1, '4a1a9578', 'Goal', 8.0, 'Head', 'de515487')
Goals Scored By Head: 3963
Execution time: 1.0844 seconds
-- Execution is: 0.12727664867816857 times faster with the index --


# 1.2 How many matches had attendees more than 12000?

In [4]:
# Create index on Attendance (do this once)
def create_index_1_2():
    cursor.execute("CREATE INDEX idx_attendance ON matches (Attendance);")
    connection.commit()
    print("Index Created: idx_attendance")

# Function to execute the query with or without index
@measure_execution_time
def MySQLQ1_2(index=False):
    # Create index if needed
    if index:
        create_index_1_2()
        print("Query with index:")
    else:
        print("Query without index:")
    
    # Define the query
    query = "SELECT * FROM matches WHERE Attendance > 12000;"
    
    # Execute the query
    cursor.execute(query)

    # Fetch all the results
    result = cursor.fetchall()

    # Print the first result (optional)
    if result:
        print(result[0])

    # Print the number of rows in the result
    print(f"Matches with attendees more than 12000: {len(result)}")

# Execute the function and measure execution time with and without index
result_no_index, execution_time_no_index = MySQLQ1_2(index=False)
result_with_index, execution_time_with_index = MySQLQ1_2(index=True)

# Compare execution times
print(f'-- Execution is: {execution_time_no_index / execution_time_with_index} times faster with the index --')




Query without index:
(2, '2022-10-09', 11, 11, 'Sun', 24671, '04eea015', '922493f3', 2, 2)
Matches with attendees more than 12000: 9166
Execution time: 0.0480 seconds
Index Created: idx_attendance
Query with index:
(2, '2022-10-09', 11, 11, 'Sun', 24671, '04eea015', '922493f3', 2, 2)
Matches with attendees more than 12000: 9166
Execution time: 0.0979 seconds
-- Execution is: 0.4906354995821486 times faster with the index --


#### 2.1 Total Goals Scoared and Inside VS Outside 16m box

In [5]:
# Create indexes (do this once)
def create_indexes_2_1():
    cursor.execute("CREATE INDEX idx_shots_outcome ON shots (Outcome);")
    cursor.execute("CREATE INDEX idx_shots_player ON shots (Player);")
    cursor.execute("CREATE INDEX idx_unique_players_id ON all_players (id);")
    connection.commit()
    print("Indexes Created")

# Function to execute the query with or without index
@measure_execution_time
def MySQLQ2_1(index=False):
    # Create indexes if needed
    if index:
        create_indexes_2_1()
        print("Query with index:")
    else:
        print("Query without index:")
    
    # SQL query with CTE and JOIN
    query = '''  
    WITH unique_players AS (
        SELECT id, name
        FROM all_players
        GROUP BY id, name
    )
    SELECT 
        p.name AS Player_name,
        SUM(CASE WHEN s.distance > 16 THEN 1 ELSE 0 END) AS goals_gt_16, 
        SUM(CASE WHEN s.distance <= 16 THEN 1 ELSE 0 END) AS goals_lt_16,
        COUNT(*) AS total_goals 
    FROM shots s
    JOIN unique_players p ON s.Player = p.id
    WHERE s.Outcome = 'Goal'
    GROUP BY p.name
    ORDER BY total_goals DESC;
    '''
    
    # Execute the query
    cursor.execute(query)
    results_from_shots = cursor.fetchall()

    return results_from_shots



# Execute the function and measure execution time with and without index
result_no_index, execution_time_no_index = MySQLQ2_1(index=False)
result_with_index, execution_time_with_index = MySQLQ2_1(index=True)

# Convert results to DataFrame for better display
df = pd.DataFrame(result_with_index, columns=['Player_name','goals_gt_16', 'goals_lt_16', 'total_goals'])
# Print the first few rows of the result
print(df.head(10))

# Compare execution times
print(f'-- Execution is: {execution_time_no_index / execution_time_with_index} times faster with the index --')



Query without index:
Execution time: 302.1917 seconds
Indexes Created
Query with index:
Execution time: 1.9367 seconds
          Player_name goals_gt_16 goals_lt_16  total_goals
0       Kylian-Mbappe          27         120          147
1  Robert-Lewandowski          15         131          146
2          Harry-Kane          21          99          120
3        Lionel-Messi          37          69          106
4       Mohamed-Salah          14          90          104
5       Ciro-Immobile          10          94          104
6       Karim-Benzema          16          88          104
7      Erling-Haaland           7          95          102
8   Wissam-Ben-Yedder           3          89           92
9    Lautaro-Martinez          15          74           89
-- Execution is: 156.03270228837354 times faster with the index --


#### 2.2 Rank teams based on average attendance at home games

In [6]:
# Create indexes (do this once)
def create_indexes_2_2():
    cursor.execute("CREATE INDEX idx_matches_home_id ON matches (home_id);")
    cursor.execute("CREATE INDEX idx_teams_id ON teams (id);")
    cursor.execute("CREATE INDEX idx_matches_attendance ON matches (Attendance);")
    connection.commit()
    print("Indexes Created")

# Function to execute the query with or without index
@measure_execution_time
def MySQLQ2_2(cursor, index=False):
    # Create indexes if needed
    if index:
        create_indexes_2_2()
        print("Query with index:")
    else:
        print("Query without index:")
    
    # SQL query to get average attendance and count of matches by home_id
    query = """
    SELECT 
        m.home_id,
        AVG(m.Attendance) AS average_attendance,
        COUNT(*) AS matches_count,
        t.name AS team_name
    FROM matches m
    INNER JOIN teams t ON m.home_id = t.id
    GROUP BY m.home_id, t.name
    ORDER BY average_attendance DESC
    """
    
    # Execute the query
    cursor.execute(query)
    results_from_matches = cursor.fetchall()  # List of tuples
    
    # Convert the result to a list of dictionaries for easier handling in Python
    results_from_matches = [
        {
            'home_id': row[0],
            'average_attendance': row[1],
            'matches_count': row[2],
            'team_name': row[3]
        } 
        for row in results_from_matches
    ]
    
    # Convert to DataFrame for easy viewing and sorting
   

    return results_from_matches

# Execute the function and measure execution time with and without index
result_no_index, execution_time_no_index = MySQLQ2_2(cursor, index=False)
result_with_index, execution_time_with_index = MySQLQ2_2(cursor, index=True)

df = pd.DataFrame(result_with_index)
df = df[['team_name', 'average_attendance', 'matches_count']]
print(df.head(10))  # Display the top 10 rows

# Compare execution times
print(f'-- Execution is: {execution_time_no_index / execution_time_with_index} times faster with the index --')



Query without index:
Execution time: 0.0270 seconds
Indexes Created
Query with index:
Execution time: 0.2027 seconds
           team_name average_attendance  matches_count
0  Manchester United         56407.2273            176
1           Dortmund         56036.0216            139
2            Benfica         54471.1429             14
3       Hamburger SV         52331.0000              3
4          Barcelona         51691.4586            157
5      Bayern Munich         50702.6525            141
6        Real Madrid         47265.5404            161
7    West Ham United         46952.0690            145
8            Arsenal         46577.0500            160
9     Internazionale         46508.7736            159
-- Execution is: 0.13322042669301154 times faster with the index --


#### 3 CREATE TABLES statements

In [7]:
for i in MySql_table_statements():
    i=i.split(',')
    for j in i:
        print(j)


    CREATE TABLE IF NOT EXISTS shots (
        ID_Index INT AUTO_INCREMENT PRIMARY KEY
 `Unnamed: 0` INT
 `Minute` INT
 `Player` VARCHAR(255)
 `Squad` VARCHAR(255)
 `xG` FLOAT
 `PSxG` FLOAT
 `Outcome` VARCHAR(255)
 `Distance` FLOAT
 `Body Part` VARCHAR(255)
 `Notes` VARCHAR(255)
 `SCA 1_Player` VARCHAR(255)
 `SCA 1_Event` VARCHAR(255)
 `SCA 2_Player` VARCHAR(255)
 `SCA 2_Event` VARCHAR(255)
 `match_id` VARCHAR(255)
    );
    

    CREATE TABLE IF NOT EXISTS matches (
        ID_Index INT AUTO_INCREMENT PRIMARY KEY
 `Unnamed: 0` INT
 `Unnamed: 1` INT
 `id` VARCHAR(255)
 `Date` VARCHAR(255)
 `league` INT
 `Round` INT
 `Day` VARCHAR(255)
 `Attendance` INT
 `Referee` VARCHAR(255)
 `Notes` VARCHAR(255)
 `played` VARCHAR(255)
 `home_id` VARCHAR(255)
 `away_id` VARCHAR(255)
 `score_away` INT
 `score_home` INT
 `position_home` INT
 `position_away` INT
 `home_capitan` VARCHAR(255)
 `away_capitan` VARCHAR(255)
 `home_xg` FLOAT
 `away_xg` FLOAT
 `home_xga` FLOAT
 `away_xga` FLOAT
 `home_formati